<a href="https://colab.research.google.com/github/Sarath-peddireddy/BreastCancerDetection/blob/main/FineTuning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip -q install accelerate -U
!pip -q install transformers[torch]
!pip -q install datasets
#Restart after installing

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
!pip install --upgrade fsspec==2024.10.0

In [3]:
!pip install gcsfs==2024.10.0

In [6]:
# Textwrp function to display the output in a better format
# This is an optional function, you can ignore it
from IPython.display import HTML, display

def wrap_display():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', wrap_display)

In [7]:
from transformers import pipeline
senti_model = pipeline(task='sentiment-analysis')
senti_model('I have grudge on one guy')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'NEGATIVE', 'score': 0.5773768424987793}]

In [10]:
senti_model2 = pipeline(task='sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment-latest")
senti_model2("waste of money")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'label': 'negative', 'score': 0.7427577376365662}]

In [11]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Amazon_Yelp_Reviews/Review_Data.csv")

# without pipeline

In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
import numpy as np
raw_text = "This is sarath the upcoming data scientist at ATLASSIAN"
encoded_input = tokenizer(raw_text, return_tensors='pt')
output = model(**encoded_input)
logits = output.logits.detach().numpy()
y_pred = np.argmax(logits)
y_pred

1

In [24]:
import numpy as np
texts=[
    "This is a great book",
    "The food was not tasty and it was very cold",
    "The weather is very good today",
]
input = tokenizer(texts,padding=True,return_tensors='pt')
output = model(**input)
logits = output.logits.detach().cpu().numpy()
y_pred = np.argmax(logits,axis =1)
y_pred

array([2, 0, 2])

# FineTuning HuggingFace Model Distilbert

In [25]:
!wget https://github.com/venkatareddykonasani/Datasets/raw/master/Bank_Customer_Complaints/complaints_v2.zip
!unzip -o complaints_v2.zip
df = pd.read_csv("/content/complaints_v2.csv")
df.head()

--2024-12-21 09:36:06--  https://github.com/venkatareddykonasani/Datasets/raw/master/Bank_Customer_Complaints/complaints_v2.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Bank_Customer_Complaints/complaints_v2.zip [following]
--2024-12-21 09:36:06--  https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Bank_Customer_Complaints/complaints_v2.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20228857 (19M) [application/zip]
Saving to: ‘complaints_v2.zip’

complaints_v2.zip   100%[===================>]  19.29M  80.2MB/s    in 0.2s    

2

,ID,product,text,label
0,0,credit_card,purchase order day shipping amount receive pro...,1
1,1,credit_card,forwarded message date tue subject please inve...,1
2,2,retail_banking,forwarded message cc sent friday pdt subject f...,1
3,3,credit_reporting,payment history missing credit report speciali...,0
4,4,credit_reporting,payment history missing credit report made mis...,0


# Use distilbert without finetuning

In [26]:
from transformers import pipeline
distilbert = pipeline(task='text-classification',model='distilbert-base-uncased')

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


In [30]:
sample = df.sample(100,random_state =42)
sample['text'] = sample['text'].apply(lambda x: " ".join(x.split()[:300]))
sample['bert_predicted'] = sample['text'].apply(lambda x:distilbert(x)[0]["label"])
sample['bert_predicted_num'] = sample['bert_predicted'].apply(lambda x: x[-1])
sample['bert_predicted_num'] = sample['bert_predicted_num'].astype(int)
sample.head()

,ID,product,text,label,bert_predicted,bert_predicted_num
156566,156566,mortgages_and_loans,penfed asking copy driver license finalizing l...,1,LABEL_0,0
1498,1498,credit_reporting,collection account removed credit report frank...,0,LABEL_0,0
134991,134991,credit_reporting,bureau falsely reporting alleged debt fdcpa se...,0,LABEL_0,0
56391,56391,mortgages_and_loans,va mortgage well fargo bank since meet conditi...,1,LABEL_0,0
9067,9067,credit_reporting,bank xxxxi credit card mine,0,LABEL_0,0


In [32]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(sample['label'],sample['bert_predicted_num'])
print(cm)
accuracy = cm.diagonal().sum()/cm.sum().astype(float)
print(f" Accuracy score before finetuning : {accuracy}")

[[47  0]
 [53  0]]
 Accuracy score before finetuning : 0.47


# Finetuning with Data

In [33]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, ClassLabel, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

In [34]:
print(sample.columns)

Index(['ID', 'product', 'text', 'label', 'bert_predicted',
       'bert_predicted_num'],
      dtype='object')


In [36]:
sample_data = Dataset.from_pandas(sample)
train_test_split = sample_data.train_test_split(test_size=0.2)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'product', 'text', 'label', 'bert_predicted', 'bert_predicted_num', '__index_level_0__'],
        num_rows: 80
    })
    test: Dataset({
        features: ['ID', 'product', 'text', 'label', 'bert_predicted', 'bert_predicted_num', '__index_level_0__'],
        num_rows: 20
    })
})

In [37]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.add_special_tokens({'pad_token': '[PAD]'} )

def tokenize_function(examples):
  return tokenizer(examples['text'], padding="max_length", truncation=True,max_length=512)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [39]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2,pad_token_id= tokenizer.eos_token_id)
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (li

In [41]:
training_args = TrainingArguments(
    output_dir = "./result_bert_custom",
    num_train_epochs=3,
    logging_dir="./logs_bert_custom",
    evaluation_strategy ='epoch',
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test']
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.686831
2,No log,0.635218
3,No log,0.597646


TrainOutput(global_step=30, training_loss=0.6415087381998698, metrics={'train_runtime': 1259.1775, 'train_samples_per_second': 0.191, 'train_steps_per_second': 0.024, 'total_flos': 31792175677440.0, 'train_loss': 0.6415087381998698, 'epoch': 3.0})

In [42]:
model_dir = "./distilbert_finetuned"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)
trainer.save_model('DistilBert_customModel_10k')

In [49]:
def make_prediction(text):
  new_complaint = text
  inputs = tokenizer(new_complaint,return_tensors='pt')
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  inputs = inputs.to(device)
  outputs = model(**inputs)
  predictions = outputs.logits.argmax(-1)
  predictions = predictions.detach().cpu().numpy()
  return(predictions)

sample_df = sample_data.to_pandas()  # Convert to pandas DataFrame

sample_df['finetuned_prediction'] = sample_df['text'].apply(lambda x:make_prediction(str(x))[0])

sample_data = Dataset.from_pandas(sample_df)

# Use .select() with random indices for sampling
import random
num_samples = 10
random_indices = random.sample(range(len(sample_data)), num_samples)
sampled_data = sample_data.select(random_indices)

sampled_data  # Display the sampled data

Dataset({
    features: ['ID', 'product', 'text', 'label', 'bert_predicted', 'bert_predicted_num', '__index_level_0__', 'finetuned_prediction'],
    num_rows: 10
})

In [51]:
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(sample_data['label'],sample_data['finetuned_prediction'])
print(cm1)
accuracy1 = cm1.diagonal().sum()/cm1.sum().astype(float)
print(f" accuarcy score after finetuning {accuracy1}")

[[36 11]
 [ 3 50]]
 accuarcy score after finetuning 0.86


#  **accuarcy score after finetuning 0.86**

# load and pre-built model to make prediction

In [52]:
#Code to donwloading the distilbert model
!gdown --id 1785J3ir19RaZP3ebbFvWUX88PMaBouro -O distilbert_finetuned_V1.zip
!unzip -o -j distilbert_finetuned_V1.zip -d distilbert_finetuned_V1

model_v1 = DistilBertForSequenceClassification.from_pretrained('/content/distilbert_finetuned_V1')


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1785J3ir19RaZP3ebbFvWUX88PMaBouro
From (redirected): https://drive.google.com/uc?id=1785J3ir19RaZP3ebbFvWUX88PMaBouro&confirm=t&uuid=d2141685-f309-4556-92ab-4405e66356fb
To: /content/distilbert_finetuned_V1.zip
100% 247M/247M [00:06<00:00, 40.2MB/s]
Archive:  distilbert_finetuned_V1.zip
  inflating: distilbert_finetuned_V1/config.json  
  inflating: distilbert_finetuned_V1/model.safetensors  
  inflating: distilbert_finetuned_V1/special_tokens_map.json  
  inflating: distilbert_finetuned_V1/tokenizer_config.json  
  inflating: distilbert_finetuned_V1/vocab.txt  


In [53]:
def make_prediction(text):
  new_complaint = text
  inputs = tokenizer(new_complaint,return_tensors='pt')
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  inputs = inputs.to(device)
  outputs = model(**inputs)
  predictions = outputs.logits.argmax(-1)
  predictions = predictions.detach().cpu().numpy()
  return(predictions)

In [1]:
sample_data_large  = df.sample(n=1000, random_state=55)
sample_data_large['finetuned_prediction'] = sample_data_large['text'].apply(lambda x:make_prediction(str(x)[:300])[0])

NameError: name 'df' is not defined

In [55]:
sample_data_large['finetuned_prediction']

,finetuned_prediction
36949,1
27628,0
138979,0
60466,0
98334,1
...,...
66079,0
122182,1
37186,1
121726,0


In [56]:
cm2 = confusion_matrix(sample_data_large['label'],sample_data_large['finetuned_prediction'])
print(cm2)
accuracy2 = cm2.diagonal().sum()/cm2.sum().astype(float)
print(f" accuarcy score after finetuning {accuracy2}")

[[386 154]
 [ 93 367]]
 accuarcy score after finetuning 0.753
